### Setup

In [179]:
import pandas as pd
import altair as alt
from functools import partial

import importlib
import utils
# Cache-Probleme umgehen
importlib.reload(utils)
from utils import create_highlight_func
from utils import highlight_rows
from utils import calc_corr
from utils import create_boxplot_with_count

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [180]:
# Farbgebung in Diagrammen
color1 = '#06507F'  # Dunkelblau
color2 = '#330241'  # Dunkellila
color3 = '#0076BF'  # Blau
color4 = '#80BADF'  # Hellblau
color5 = '#E7DDFF'  # Helllila


### Data Load

In [181]:
path_data = 'https://raw.githubusercontent.com/mm391-030401/project/refs/heads/main/data/processed/'
file_data= 'data_final.csv'

data = pd.read_csv(path_data + file_data, sep=',', encoding='utf-8')

# Obwohl diese Spalten bereits im Notebook "convert_columns" zu Strings konvertiert wurden, wurden sie beim erneuten Einlesen wieder als Integer erkannt
# Allerdings ist es sinnvoll, diese Spalten nicht als Integer (diskret) zu betrachten 
# sondern als ordinale Variablen (Rechenoperationen sind nicht sinnig für die Zeitangaben)
to_str_cols = ['monat', 'monat_jahr','monat_monat', 'kohorte', 'kohorte_jahr', 'kohorte_monat']

# Konvertierung der Spalten in string 
for col in to_str_cols: 
    data[col] = data[col].astype(str)

### Variable List

In [182]:
y_label = 'retentionrate'
x_float = 'rabatt_indexiert' # die anderen kontinuierlichen Variablen gehören zur Response Variable
list_numeric = data.select_dtypes(include = 'number').columns.to_list()
list_category = data.select_dtypes(include = ['object', 'bool']).columns.to_list()

### Data Structure

In [183]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 703 entries, 0 to 702
Data columns (total 15 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   monate_seit_einfuehrung_programm_kohorte  703 non-null    int64  
 1   monat                                     703 non-null    object 
 2   monat_jahr                                703 non-null    object 
 3   monat_monat                               703 non-null    object 
 4   monat_jahreszeit                          703 non-null    object 
 5   kohorte                                   703 non-null    object 
 6   kohorte_jahr                              703 non-null    object 
 7   kohorte_monat                             703 non-null    object 
 8   kohorte_jahreszeit                        703 non-null    object 
 9   erster_monat_kohorte_fg                   703 non-null    bool   
 10  monate_seit_existenz_kohorte          

In [184]:
data.shape

(703, 15)

In [185]:
data.head()

,monate_seit_einfuehrung_programm_kohorte,monat,monat_jahr,monat_monat,monat_jahreszeit,kohorte,kohorte_jahr,kohorte_monat,kohorte_jahreszeit,erster_monat_kohorte_fg,monate_seit_existenz_kohorte,kohortengroesse_indexiert,identifizierte_kunden_indexiert,rabatt_indexiert,retentionrate
0,-2,201408,2014,8,Sommer,201408,2014,8,Sommer,True,0,0.41,0.41,2.54,100.000000
1,-2,201409,2014,9,Herbst,201408,2014,8,Sommer,False,1,0.41,0.40,7.14,97.560976
2,-2,201410,2014,10,Herbst,201408,2014,8,Sommer,False,2,0.41,0.39,9.28,95.121951
3,-2,201411,2014,11,Herbst,201408,2014,8,Sommer,False,3,0.41,0.38,3.22,92.682927
4,-2,201412,2014,12,Winter,201408,2014,8,Sommer,False,4,0.41,0.38,7.15,92.682927


In [186]:
data.tail()

,monate_seit_einfuehrung_programm_kohorte,monat,monat_jahr,monat_monat,monat_jahreszeit,kohorte,kohorte_jahr,kohorte_monat,kohorte_jahreszeit,erster_monat_kohorte_fg,monate_seit_existenz_kohorte,kohortengroesse_indexiert,identifizierte_kunden_indexiert,rabatt_indexiert,retentionrate
698,32,201707,2017,7,Sommer,201706,2017,6,Sommer,False,1,18.36,9.45,17.88,51.470588
699,32,201708,2017,8,Sommer,201706,2017,6,Sommer,False,2,18.36,8.40,10.59,45.751634
700,33,201707,2017,7,Sommer,201707,2017,7,Sommer,True,0,17.49,17.49,23.76,100.000000
701,33,201708,2017,8,Sommer,201707,2017,7,Sommer,False,1,17.49,8.54,11.85,48.827902
702,34,201708,2017,8,Sommer,201708,2017,8,Sommer,True,0,15.13,15.13,14.28,100.000000


### Analysis

#### Descriptive Statistics

In [187]:
data[list_numeric].describe().T

,count,mean,std,min,25%,50%,75%,max
monate_seit_einfuehrung_programm_kohorte,703.0,10.000000,8.838049,-2.000000,3.000000,8.000000,16.000000,34.00
monate_seit_existenz_kohorte,703.0,12.000000,8.838049,0.000000,5.000000,10.000000,18.000000,36.00
kohortengroesse_indexiert,703.0,47.926743,77.462511,0.410000,23.110000,24.850000,33.930000,375.13
identifizierte_kunden_indexiert,703.0,28.301565,53.452328,0.310000,10.890000,12.320000,21.215000,375.13
rabatt_indexiert,703.0,99.719090,257.969362,2.260000,21.500000,33.020000,61.045000,2570.59
retentionrate,703.0,55.442768,15.491696,35.752785,46.394922,49.102845,57.336771,100.00


In [188]:
data[list_category].describe().T

,count,unique,top,freq
monat,703,37,201708,37
monat_jahr,703,4,2016,282
monat_monat,703,12,8,76
monat_jahreszeit,703,4,Sommer,217
kohorte,703,37,201408,37
kohorte_jahr,703,4,2015,318
kohorte_monat,703,12,8,76
kohorte_jahreszeit,703,4,Herbst,207
erster_monat_kohorte_fg,703,2,False,666


- `kohortengroesse_indexiert` sowie `identifizierte_kunden_indexiert` zeigen eine hohe Diskrepanz zwischen dem 3. Quantil und dem Maximalwert. Es gibt also wenige Kohorten, die eine sehr hohe Anzahl an Kunden ausweisen. Je höher die anfängliche Kundenanzahl desto höher dann auch die Anzahl der möglichen Kunden, die sich jeden Monat identifizieren können.
- Ebenfalls gibt es bestimmte Kohorten-Monats-Kombinationen, die einen hohen Rabatt erhalten haben. 
- Die maximale Retentionrate liegt bei 100. Im ersten Monat der Kohorte (dann, wenn die Kohorte entsteht) ist der Wert von Spalte `kohortengroesse_indexiert` und `identifizierte_kunden_indexiert` immer gleich. Hier liegt also die Retentionrate bei 100. Der starke Kundenrückgang nach dem ersten Monat einer Kohorte zeigt sich in den Werten von den Quantilen. Die Differenz zwischen 3. Quantil und Maximalwert ist ebenfalls hoch.
- Keine der Spalten enthält missing values.

#### Numeric Data

In [189]:
# Histogramm
hist = alt.Chart(data).mark_bar(color=color1).encode(
    x=alt.X(y_label, bin=alt.Bin(maxbins=30), title=y_label),
    y=alt.Y('count()', title='Anzahl'),
    tooltip=alt.Tooltip(value='Histogramm') 
).properties(
    title='Übersicht Verteilung Response Variable',
    width=600,
    height=400
).interactive()

# Mittelwert-Linie
mean = alt.Chart(data).mark_rule(color=color4).encode(
    x=alt.X(f'mean({y_label}):Q'),
    size=alt.value(5),
    tooltip=alt.Tooltip(value='Mittelwert') 
)

# Median-Linie
median = alt.Chart(data).mark_rule(color=color3).encode(
    x=alt.X(f'median({y_label}):Q'),
    size=alt.value(5),
    tooltip=alt.Tooltip(value='Median') 
)

# Dichtekurve
density = alt.Chart(data).transform_density(
    y_label, 
    as_=[y_label, 'density'],
    bandwidth=1  # Hier eine geeignete Bandbreite
).mark_line(color=color2).encode(
    x=alt.X(f'{y_label}:Q', title=y_label),
    y=alt.Y('density:Q', title='Dichte'),
    tooltip=alt.Tooltip(value='Dichtekurve'),
    size = alt.value(5)
).properties(
    width=600,
    height=400
)

# Kombination der Charts mit sekundärer y-Achse für die Dichtekurve
combined_chart = alt.layer(
    hist,
    density.encode(y=alt.Y('density:Q', axis=alt.Axis(title='Dichte')))
).resolve_scale(
    y='independent'
) + mean + median

combined_chart



alt.LayerChart(...)


**Interpretation der Response-Variable**  
Die Verteilung der Response-Variable zeigt deutliche Eigenschaften, die für die weitere Modellierung berücksichtigt werden sollten. Das Histogramm verdeutlicht, dass die Daten eine asymmetrische Verteilung mit einer starken Rechtsschiefe aufweisen. Der Großteil der Werte konzentriert sich im Bereich von 45 bis 55, was auf eine hohe Dichte in diesem Bereich hinweist. Gleichzeitig gibt es einen kleineren Anteil an Beobachtungen in den höheren Bereichen von 70 bis 100, was auf mögliche Ausreißer oder spezifische Subgruppen in den Daten hindeutet.

Die Lage der beiden zentralen Maße – Median (dunkle vertikale Linie) und Mittelwert (helle vertikale Linie) – unterstreicht die positive Schiefe der Verteilung. Der Median liegt deutlich links vom Mittelwert, was darauf zurückzuführen ist, dass die höheren Werte den Mittelwert nach rechts ziehen.

Die Dichtekurve gibt zusätzlich interessante Einblicke in die Struktur der Daten. Neben einem klaren Hauptgipfel im Bereich von 45 bis 50 zeigt die Kurve kleinere Peaks in den Bereichen von 80 bis 95. Diese könnten auf unterschiedliche Gruppen oder Cluster innerhalb der Daten hinweisen, wie z. B. verschiedene Kundensegmente oder Bedingungen, die unterschiedliche Retentionsraten bewirken.

Implikationen für die Modellierung  
Die Analyse der Verteilung liefert mehrere Hinweise für die Vorbereitung der Daten:
- Schiefe der Verteilung:  
Die starke positive Schiefe könnte die Modellleistung beeinflussen. Eine Transformation der Daten, wie beispielsweise eine logarithmische Transformation, könnte dazu beitragen, die Verteilung zu symmetrisieren.
- Ausreißer:  
Die Werte im Bereich 80–100 sollten genauer untersucht werden. Es könnte sinnvoll sein, diese Ausreißer zu behandeln oder separat zu modellieren.

In [190]:
# Histogramm
hist_x = alt.Chart(data).mark_bar(color=color1).encode(
    x=alt.X(x_float, bin=alt.Bin(maxbins=50), title=x_float),
    y=alt.Y('count()', title='Anzahl'),
    tooltip=alt.Tooltip(value='Histogramm') 
).properties(
    title='Übersicht Verteilung X-Variable kontinuierlich',
    width=600,
    height=400
).interactive()

# Dichtekurve
density_x = alt.Chart(data).transform_density(
    x_float, 
    as_=[x_float, 'density'],
    bandwidth=1  # Hier eine geeignete Bandbreite
).mark_line(color=color2).encode(
    x=alt.X(f'{x_float}:Q', title=x_float),
    y=alt.Y('density:Q', title='Dichte'),
    tooltip=alt.Tooltip(value='Dichtekurve'),
    size = alt.value(2)
).properties(
    width=600,
    height=400
)

# Kombination der Charts mit sekundärer y-Achse für die Dichtekurve
combined_chart_x = alt.layer(
    hist_x,
    density_x.encode(y=alt.Y('density:Q', axis=alt.Axis(title='Dichte')))
).resolve_scale(
    y='independent'
) 

combined_chart_x


alt.LayerChart(...)

Der Rabatt ist stark rechtsschief. Sollte er in das Modell inkludiert werden, so ist eventuell eine Transformation notwendig.

In [191]:
alt.Chart(data).mark_circle(color = color1).encode(
    x=alt.X(alt.repeat("column"), 
            type='quantitative',
            scale=alt.Scale(zero=False)
            ),
    y=alt.Y(alt.repeat("row"), 
            type='quantitative',
            scale=alt.Scale(zero=False)
             )
).properties(
    width=150,
    height=150
).repeat(
    row=list_numeric,
    column=list_numeric
)

alt.RepeatChart(...)

Da die identifizierten Kunden und die Kohortengröße zur Berechnung der Response Variable benötigt werden, eignen sie sich nicht als Predictor. Der Rabatt könnte einen Einfluss haben. Das Muster wird nachfolgend genauer untersucht. `monate_seit_existenz_kohorte` und `monate_seit_einfuehrung_programm_kohorte` sollten eher in Boxplots dargestellt werden, da sie sich als diskrete Variablen nicht so gut für einen Scatterplot eignen.

In [192]:
# Tooltip Variable für die folgenden Diagramme
tooltip = ['kohorte', 'monat_monat']

In [193]:
alt.Chart(data).mark_circle(color=color1).encode(
    x = alt.X(x_float),
    y = alt.Y(y_label),
    tooltip = tooltip
).interactive()

alt.Chart(...)

Die rechten Ausreißer stammen ausschließlich aus der Kohorte 202410. Die Kohorte beansprucht den meisten Rabatt. Damit ein möglicher Zusammenhang nicht nur durch die oberen Ausreißer erscheint, wird sich der Scatterplot nochmal nur für diese Kohorte und einmal für alle ohne diese Kohorte angeschaut.

In [194]:
data_201410 = data[data['kohorte'] == '201410']

alt.Chart(data_201410).mark_circle(color=color1).encode(
    x = alt.X(x_float),
    y = alt.Y(y_label),
    tooltip = tooltip
).interactive()

alt.Chart(...)

In [195]:
data_wo_201410 = data[data['kohorte'] != '201410']

alt.Chart(data_wo_201410).mark_circle(color=color1).encode(
    x = alt.X(x_float),
    y = alt.Y(y_label),
    tooltip = tooltip
).interactive()

alt.Chart(...)

Nach Betrachtung der Daten wirkt es fast so, als ob jede Kohorte seinem eigenem Muster folgt. Daher wird einmal für jede Kohorte ein Scatterplot erstellt.

In [196]:
# Liste der einzigartigen Werte in der Spalte 'kohorte'
unique_kohorten = data['kohorte'].unique()

In [197]:
# Leere Liste, um die Charts zu speichern
charts = []

# Schleife über alle einzigartigen Werte in 'kohorte'
for kohorte in unique_kohorten:
    # Filtert den DataFrame nach der aktuellen Kohorte
    df_filtered = data[data['kohorte'] == kohorte]
    
    # Erstellt das Diagramm
    chart = alt.Chart(df_filtered).mark_circle(color=color1).encode(
        x=alt.X(x_float),
        y=alt.Y(y_label),
        tooltip=tooltip + [x_float, y_label]
    ).interactive()
    
    # Fügt eine Überschrift hinzu
    chart = chart.properties(
        title=f'Kohorte {kohorte}',
        width = 150,
        height = 150
    )
    
    # Fügt das Diagramm der Liste hinzu
    charts.append(chart)

# Erstellt ein Rasterlayout aus den Diagrammen
grid_chart = alt.concat(*charts, columns=6)

grid_chart.display()

alt.ConcatChart(...)

Bei den meisten Kohorten scheint eine positive Korrelation zwischen dem Rabatt und der Retentionrate zu exisitieren. Dies wird nun mit einer Berechnung der Korrelation je Kohorte überprüft.

In [198]:
# Leere Liste, um die Ergebnisse zu speichern
korrelations_liste = []

# Schleife über alle einzigartigen Werte in 'kohorte'
for kohorte in unique_kohorten:
    # Filtert den DataFrame nach der aktuellen Kohorte
    df_filtered = data[data['kohorte'] == kohorte]

    # Berechnet die Korrelation (Es wird Spearman verwendet, da bereits in der deskriptven Statistik sowie in den Scatterplots deutlich wird, dass es Ausreißer gibt.)
    # Spearman ist besser im Umgang mit Ausreißern im Vergleich zu Pearson
    korrelation = df_filtered[[y_label, x_float]].corr(method='spearman').iloc[0, 1]
    
    # Fügt die Ergebnisse zur Liste hinzu
    korrelations_liste.append({
        'Kohorte': kohorte,
        'Korrelation': korrelation
    })

# Erstellt einen DataFrame aus der Liste
korrelations_df = pd.DataFrame(korrelations_liste)

# Anzahl der Zeilen
n = len(korrelations_df)

# Aufteilen des DataFrames in zwei Hälften
first_half = korrelations_df.iloc[:n//2]
second_half = korrelations_df.iloc[n//2:]

# Kombiniert die beiden Hälften in einem DataFrame mit zwei Spalten
# So soll das Df dann später übersichtlicher und kompakter dargestellt werden
combined_df = pd.concat([first_half.reset_index(drop=True), second_half.reset_index(drop=True)], axis=1)
combined_df.columns = ['Kohorte_Zeile1', 'Korrelation_Zeile1', 'Kohorte_Zeile2', 'Korrelation_Zeile2']

# Transponiert das df
combined_df = combined_df.T

# Anwenden der Farbformatierung und Runden der Werte auf zwei Nachkommastellen
highlight_func = create_highlight_func(combined_df, color1, color2)
styled_df = combined_df.style.apply(highlight_func, axis=1).format(precision=2)

# Setzen der Schriftfarbe auf Weiß für den gesamten DataFrame
styled_df = styled_df.set_table_styles(
    [{'selector': 'td', 'props': [('color', 'white')]}],  # Schriftfarbe auf Weiß setzen
    overwrite=False
)

styled_df


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Kohorte_Zeile1,201408,201409,201410,201411,201412,201501,201502,201503,201504,201505,201506,201507,201508,201509,201510,201511,201512,201601,nan
Korrelation_Zeile1,0.08,0.19,0.65,0.64,0.69,0.42,0.30,0.22,0.12,0.06,0.16,0.02,-0.16,-0.29,-0.36,-0.32,-0.06,-0.14,nan
Kohorte_Zeile2,201602,201603,201604,201605,201606,201607,201608,201609,201610,201611,201612,201701,201702,201703,201704,201705,201706,201707,201708
Korrelation_Zeile2,0.04,-0.05,0.21,0.24,0.46,0.43,0.69,0.80,0.91,0.94,0.83,0.93,0.89,0.94,0.90,1.00,1.00,1.00,nan


In [199]:
calc_corr(data, y_label, x_float)

Korrelation zwischen retentionrate und rabatt_indexiert beträgt: 0.24


Die Korrelation zwischen dem Rabatt und der Retentionrate schwankt je nach Kohorte. Die Korrelation insgesamt beträgt auch nur 0.24 und ist damit eher schwach. Es ist fraglich, ob der Rabatt tatsächlich ein geeigneter Predictor ist.  
Daher wird vorgeschlagen erst die Auswahl der Zeitvariablen zu optimieren und dann am Schluss den Rabatt als Feature mitzugeben und zu schauen, ob es das Modell positiv beeinflusst.

In [200]:
create_boxplot_with_count(data, y_label, 'monate_seit_einfuehrung_programm_kohorte', color1)

alt.LayerChart(...)

`monate_seit_einfuehrung_programm_kohorte` gibt an, an welchem Zeitpunkt des Lebenszyklus des Programms die Kohorte dazugekommen ist. Die Kohorten, die schon relativ früh ins Programm eingetreten sind (weiter links auf der X-Achse), zeigen eine eher höhere Retention Rate. Auch die sehr späten Kohorten zeigen eine hohe Retention Rate. Dies liegt jedoch daran, dass nur wenige Werte in die Verteilung eingeflossen sind (siehe blaue Zahlen im Diagramm). Da der erste Monat immer eine Retentionrate von 100 aufweist (gut daran zu sehen, dass bei jedem Wert auf der x-Achse mindestens eine 100 dabei ist), wirkt es so, als ob die späteren Kohorten eine hohe Retention haben. Würde man diese Kohorten im weiteren Zeitverlauf beobachten, so würde die Retentionrate analog der anderen Kohorten ebenfalls sinken und die Box verkleinern und nach unten verschieben.

In [201]:
create_boxplot_with_count(data, y_label, 'monate_seit_existenz_kohorte', color1)

alt.LayerChart(...)

`monate_seit_existenz_kohorte` gibt an, wie lang die aktuelle Kohorte schon teil des Programms ist. Daher ist bei Wert 0 die Retentionrate auch ausschließlich 100, da sie den ersten Monat der Kohorte angibt und auch alle 37 Kohorten beinhaltet. Der rapide Abstieg von 0 auf 1 zeigt das normale Verhalten der Kunden, dass die, die im ersten Monat da waren nicht alle im Folgemonat zurückkehren (Median von Monat 1, also einen Monat nach Entstehung der Kohorte, liegt bei 59,5). Bis Monat 9 ist ein Abstieg der Retentionrate zu beobachten (Gemessen am Median). Danach steigt die Retention tendenziell wieder. Dies zeigt zwei Effekte: Es findet keine Kundenabwanderung im großen Stil mehr statt. Gleichzeitig kommen eher die zurück, die im Monat 0 sich das erste mal identifiziert haben (Schwankendes Einkaufsverhalten, welches der Kunde über die App identifiziert). Zusätzlich sind immer weniger Kohorten vorhanden, die den Mittelwert beeinflussen, je stärker diese Variable ansteigt (Wert 36 zeigt ausschließlich die älteste Kohorte, siehe blaue Zahlen im Diagramm). Aus der Folgebetrachtung der Variable `kohorte` geht hervor, dass frühere Kohorten eine tendenziell höhere Retention zeigen. Der Doppeleffekt führt zu einer zunehmenden Steigerung der Retentiorate.

**Überprüfung Korrelation**  
Es wird die Spearman-Korrelation zwischen den beiden diskreten Variablem und der Response Variable berechnet aufgrund dem besseren Umgang mit Ausreißern.

In [202]:
calc_corr(data, y_label, 'monate_seit_einfuehrung_programm_kohorte')

Korrelation zwischen retentionrate und monate_seit_einfuehrung_programm_kohorte beträgt: -0.52


In [203]:
calc_corr(data, y_label, 'monate_seit_existenz_kohorte')

Korrelation zwischen retentionrate und monate_seit_existenz_kohorte beträgt: -0.07


In [204]:
calc_corr(data, 'monate_seit_einfuehrung_programm_kohorte', 'monate_seit_existenz_kohorte')

Korrelation zwischen monate_seit_einfuehrung_programm_kohorte und monate_seit_existenz_kohorte beträgt: -0.47


`monate_seit_existenz_kohorte` korreliert nicht mit der Response Variable. `monate_seit_einfuehrung_programm_kohorte` zeigt allerdings eine Korrelation und könnte ein guter Prediktor für das Modell sein.

In [210]:
create_boxplot_with_count(data, y_label, 'monate_seit_einfuehrung_programm_kohorte', color1) & create_boxplot_with_count(data, y_label, 'kohorte', color1) 

alt.VConcatChart(...)

Es ist zu beachten, dass `monate_seit_einfuehrung_programm_kohorte` und `kohorte` auf die gleiche Weise die Daten abbilden. Wird also `monate_seit_einfuehrung_programm_kohorte` in das Modell inkludiert, so sind alle anderen Variablen, die Auskunft über die Entstehung der Kohorte geben grundsätzlich hinfällig (`kohorte`, `kohorte_monat`, `kohorte_jahr`, `kohorte_jahreszeit`).  
Die einzige sinnvolle Ergänzung bieten `kohorte_monat` und `kohorte_jahreszeit` da sie keine Aussage über das Alter der Kohorte machen sondern eher eine Aussage darüber treffen, in welcher Saison die Kohorte dazugekommen ist. Hält die Annahme stand, dass Kundenverhalten saisonal ist, so könnte die Saison, in der eine Kohorte entsteht, das Verhalten der Kohorte beeinflussen. Es ist zu prüfen, ob einer der beiden Variablen das Modell verbessert.

#### Categoric Data

In [209]:
charts_cat = []

for cat in list_category:
    # Prüfen, ob die Kategorie numerisch sortiert werden soll
    if cat in ['monat_monat', 'kohorte_monat']:  # Nur für diese Kategorien numerische Sortierung
        sort_order = sorted(data[cat].unique(), key=lambda x: float(x))
    else:
        # Wenn die Kategorie nicht in der Liste ist, lasse die Sortierung standardmäßig
        sort_order = None

    # Erstellen des Diagramms (Median statt Mean)
    chart = alt.Chart(data).mark_bar(color=color1).encode(
        x=alt.X(f'median({y_label}):Q', title=y_label), 
        y=alt.Y(f'{cat}', title=cat, sort=sort_order),
        tooltip=[
            alt.Tooltip(f'{cat}:N', title='Kategorie'),  # Kategorie anzeigen
            alt.Tooltip(f'median({y_label}):Q', title='Median', format=".2f"),  # Median mit 2 Dezimalstellen
            alt.Tooltip(f'count():Q', title='Anzahl Zeilen für Median')  # Wie viele Werte sind im Median enthalten?
        ]
    ).properties(
        title=f'Median {y_label} je {cat}',
        width=300,
        height=300
    ).interactive()
    
    charts_cat.append(chart)

# Kombinieren der Charts in ein Grid
grid_chart_cat = alt.concat(*charts_cat, columns=3)
grid_chart_cat.display()


alt.ConcatChart(...)

- Die Variable `erster_monat_kohorte_fg` scheint relevant für das Modell zu sein. So wird deutlich, dass der erste Monat einer Kohorte, wo die Retention immer bei 100 liegt, nicht die Standardretention ist.
- Ebenfalls wird deutlich, dass beiden zeitliche Faktoren (Entstehung der Kohorte und Zeitverlauf des Einkaufsverhaltens) in das Modell inkludiert werden sollten. Im Zeitverlauf (`monat`) wird die Retention eher schlechter, während frühe als auch spätere Kohorten (`kohorte`) eher eine bessere Retention zeigen. Es besteht ein Unterschied in der Reaktion der Retention zwischen dem normalen Zeitverlauf in Monaten vs. der Monate der Entstehung der Kohorten. Das "Level" der Zeit (also braucht man die Granularität von `monat` und `kohorte` oder reicht z.B. aich `kohorte_monat` und `monat_monat`) sollte in mehreren Testläufen optimiert werden.
- Zu beachten ist hierbei die zuvor gewonnene Erkenntnis über die Variable `monate_seit_einfuehrung_programm_kohorte`.


**Boxplots der kategorischen Variablen für tiefere Insights**

In [207]:
boxplots_cat = []

# For-Schleife zur Erstellung der Boxplots
for cat in list_category:
    # Prüfen, ob die Variable numerisch behandelt werden soll
    if cat in ['monat_monat', 'kohorte_monat']: 
        # Berechnung der unteren und oberen Grenzen für x_limits
        lower_limit = data[cat].astype(int).min()
        upper_limit = data[cat].astype(int).max()

        # Erstellen des Boxplots mit x_limits
        result = create_boxplot_with_count(data, y_label, cat, color1, x_type='Q', x_limits=(lower_limit - 0.25, upper_limit + 0.25))
    else:  # Falls nicht numerisch
        # Erstellen des Boxplots ohne x_limits
        result = create_boxplot_with_count(data, y_label, cat, color1)

    boxplots_cat.append(result)

grid_boxplot_cat = alt.concat(*boxplots_cat, columns=1)
grid_boxplot_cat.display()


alt.ConcatChart(...)

### Fazit für die Modellierung  
- Um die Ausreißer korrekt zu behandeln, sollte unbedingt die Variable `erster_monat_kohorte_fg` mit in die Modellierung einfließen.
- Sowohl die Variable `kohorte` als auch `monat` ist von Relevanz.  
Bei der `kohorte` ist es eher wichtig, auf den Zeitpunkt der Entstehung (Early vs. Late Adopters) zu achten. Die Variable `monate_seit_einfuehrung_programm_kohorte` zeigt dies gut auf. Um mögliche Saisonalitäten abzufangen, ist es möglich `kohorte_monat` oder `kohorte_jahreszeit` ebenfalls zu inkludieren. `kohorte` und `kohorte_jahr` sind allerdings nicht notwendig.
Bei `monat` ist es eher wichtig, die Saisonalität, die die Retentionrate beeinflussen kann, abzubilden. Es sollte daher mindestens `monat_jahreszeit` inkludiert werden.
- `rabatt_indexiert` scheint kein verlässlicher Prediktor zu sein. Er sollte zum Schluss hinzugefügt werden.
- Es ist zu prüfen ob eine logarithmische Transformation der Response-Variable das Modell verbessert (aufgrund Rechtsschiefe). Falls `rabatt_indexiert` in das Modell einbezogen wird, sollte diese auch transformiert werden (ebenfalls Rechtsschiefe). Da die Rechtsschiefe hauptsächlich durch die Retentionrate 100 ausgelöst wird und diese durch die Variable `erster_monat_kohorte_fg` erklärt wird, ist die Transformation eventuell nicht notwendig.  

**Vorgehensweise Modellierung**
- Inkludieren von `erster_monat_kohorte_fg` und `monate_seit_einfuehrung_programm_kohorte`
- Test `monat_monat` vs. `monat_jahreszeit`
- Test `kohorte_monat` vs. `kohorte_jahreszeit`
- Test ob `rabatt_indexiert` Modell verbessert
- Einmal nochmal alles mit logarithmischer Transformation
- Andere Modelle testen (z.B. Random Forest)